In [10]:
from music21 import *
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pickle
import glob

In [11]:
notes=numpy.array([])

for midfile in glob.glob('../data/sonata14/noduration/*'):
    with open(midfile, 'rb') as path:

        notes = numpy.append(notes,pickle.load(path))
        
print(notes)

['C#2' 'G#2' '49, 44|chord' ..., 'C#2' '61, 52, 56, 44, 49, 37|chord'
 '61, 52, 56, 44, 49, 37|chord']


In [12]:
chars = sorted(list(set(notes)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(notes)
n_vocab = len(chars)
print ("Total Characters: ")
print(n_chars)
print ("Total Vocab: ")
print(n_vocab)

Total Characters: 
5715
Total Vocab: 
753


In [13]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 28
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = notes[i:i + seq_length]
    seq_out = notes[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print( "Total Patterns: ") 
print(n_patterns)

Total Patterns: 
5687


In [14]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
print(y.shape[1])

753


In [15]:
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
filename = "w195-0.1066-moonlightfull.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print(pattern)

[710, 746, 706, 711, 710, 746, 706, 711, 443, 710, 736, 696, 710, 736, 696, 404, 715, 736, 696, 431, 710, 736, 696, 490, 725, 736, 746, 725]


In [18]:
offset = 0.5
aInterval = interval.Interval(-0.1)
stream2 = stream.Stream()
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(result)
    if(result[-6:len(result)]=='|chord'):
        pnote = result.replace("|chord","")
        pnote = pnote.split(',')
        print(i)
        mychord =''
        for p in pnote:
            p1 = pitch.Pitch()
            p1.midi=float(p)
            mychord +=p1.nameWithOctave.replace("-","")
            mychord +=' '
            firstchord = chord.Chord(mychord)##perfect
            
        stream2.append(firstchord)
        

    else:


        npitch = result
        
        notez = note.Note(npitch)
        if('-' in npitch):
            notez = notez.transpose(aInterval)
            
        notez.offset=offset
        stream2.append(notez)
        offset += 0.5


    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    

stream2.write('midi', fp='saida01.mid')

F#4
G#4
E-4
F#4
G#4
75, 36, 44, 48|chord
5
E-4
F#4
G#4
76, 37, 44, 49|chord
9
E4
G#4
C#5
E4
G#4
C#5
75, 42, 30|chord
16
E-4
F#4
A4
73, 31, 43|chord
20
C#4
E4
B-4
C5
32, 44|chord
25
C4
E-4
G#4
C4
E-4
A4
C4
E-4
F#4
C4
E-4
44, 32|chord
37
C4
E-4
G#3
C4
E-4
A3
C4
E-4
F#3
C4
E-4
E3
32, 44|chord
50
E4
G#4
C#5
E4
G#4
E5
E4
G#4
C#5
E4
G#4
32, 44|chord
62
E3
G#3
C#4
E3
G#3
E4
E3
G#3
C#4
E3
G#3
E-3
44, 32|chord
75
A3
F#3
C4
A3
E-4
C4
F#4
E-4
A4
F#4
C5
E3
44, 32|chord
88
C#4
G#3
E4
C#4
G#4
E4
C#5
G#4
E5
C#5
G#4
C#4
32, 44|chord
101
G4
E4
B-4
G4
C#5
B-4
E5
C#5
G5
E5
B-5
F#4
32, 44|chord
114
C5
A4
E-5
C5
F#5
E-5
A5
F#5
C6
A5
E-6
C6
F#5
A5
E-5
F#5
C5
E-5
A4
C5
F#4
A4
E-4
F#4
C4
E-4
A3
C4
F#3
A3
E-3
F#3
C#3
F#3
A3
C3
32, 44|chord
151
F#3
G#3
A3
G#3
F#3
E-3
F#3
A3
C#3
F#3
A3
C3
32, 44|chord
164
F#3
G#3
A3
G#3
F#3
D3
F#3
A3
C#3
F#3
A3
48, 32, 44|chord
176
F#3
G#3
A3
G#3
F#3
C#3
45, 33|chord
183
E3
C#4
C#3
E3
C#4
E-3
30, 42|chord
190
A3
C#4
E-3
A3
C#4
E-3
44, 32|chord
197
G#3
C4
E-3
F#3
C4
E3
37, 49, 44

'saida01.mid'